<h1><b>Statistique en Bioinformatique : </b> TME5 et 6 </h1>
<br>
L’objectif de ce TME est: 
<br>
<ul>
<li> implémenter l'algorithme de Viterbi et l'estimation des paramètres (en utilisant le Viterbi training)
pour l'exemple du occasionally dishonest casino.   </li> 
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p>**Soumission**</p>
<ul>
<li>Renomer le fichier TME5_6.ipynb pour NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Envoyer par email à edoardo.sarti@upmc.fr, l’objet du email sera [SBAS-2019] TME5 (deadline 19/03/2018 23:59)</li>
</ul>
</div>

Nom etudiant 1 : Alex YE
<br>
Nom etudiant 2 :
<br>

<h3>Introduction</h3>
Un casino parfois malhonnête (occasionally dishonest casino) utilise 2 types de pieces : fair et unfair. <br>
La matrice de transition entre les états cachés est:<br>
${\cal S}=\{F,U\}$ (fair, unfair):
$$
p = \left(
\begin{array}{cc}
0.99 & 0.01\\
0.05 & 0.95
\end{array}
\right)\ ,
$$

les probabilités d'éemission des symboles 
${\cal O} = \{H,T\}$ (head, tail):
\begin{eqnarray}
e_F(H) =  0.5 &\ \ \ \ &
e_F(T) = 0.5 \nonumber\\
e_U(H) = 0.9 &\ \ \ \ &
e_U(T) = 0.1 \nonumber
\end{eqnarray}

<br> Et la condition initiale $\pi^{(0)} = (1,0)$ (le jeux commence toujours avec le pieces juste (fair).

<b>Exercice 1</b>:
<u>Simulation</u>: Ecrire une fonction qui simule $T$ jets de pieces. 
La fonction renverra un tableau à deux colonnes correspondant 
aux valeurs simulées pour les états cachés $X_t$ 
(type de dés utilisée, “F” ou “U”) et aux symboles observées $Y_t$ 
(résultat du jet de dés, “H” ou “T”). On simulera une séquence
de longueur 2000 qu'on gardera pour les applications ultérieures.


In [1]:
import numpy as np
import matplotlib.pyplot as plt


S = { 0:'F',1 :'U'}
Pij = np.array([[0.99, 0.01], 
                [0.05, 0.95]])

O = {0:'H', 1: 'T'}
Eij = np.array([[0.5, 0.5], 
                [0.9, 0.1]]) #ça aurait dû être Eio

# Condition initiale
pi0=np.array([0.999,0.001])

T = 2000

In [2]:
import random

# Fonction qui simule T jets de pieces
def toss(times, pi, trans, emiss):
    #il n'y a toujours que 2 états possibles, si ce n'est pas le premier c'est forcément l'autre. 
    hid = np.random.rand(times)
    obs = np.random.rand(times)
    state = []
    
    #initial state
    curr_state = 0
    if(hid[0] < pi[0]):
        state.append(0)
        curr_state = 0
    else:
        state.append(1)
        curr_state = 1
        
    #state transition
    for i in range(1,times):
        if(hid[i] < trans[curr_state][0]):
            state.append(0)
            curr_state = 0
        else:
            state.append(1)
            curr_state = 1
    
    #emission from state
    tir = []
    for i in range(times):
        if(obs[i]< emiss[state[i]][0]):
            tir.append(0) #on ne vas pas mettre F ou U, pour simplifier les calculs suivants.
        else:
            tir.append(1)
    
    return np.array(state), np.array(tir)
    
states, sample = toss(T, pi0, Pij, Eij)
print(sample)
print(states)

[0 0 1 ... 1 0 0]
[0 0 0 ... 0 0 0]


<b>Exercice 2</b>: <u>Algorithme de Viterbi </u>: Ecrire une fonction qui permet
de déterminer la séquence $(i^\star_t)_{t=0:T}$ d'états cachés
plus probable, ansi que sa probabilité. Pour tester votre fonction utiliser le résultat de la 
simulation (2éme colonne) de la question 1. Comparer $(i^\star_t)_{t=0:T}$ avec
les vrais états cachés (1ère colonne de la simulation). 


In [11]:
# unused 
def forward(seq, trans, emiss):    
    alpha = np.zeros((seq.size,len(emiss)))
    alpha[0][:] = 1
    for i in range(len(trans)): #for all state
        for t in range(1,seq.size): #for all observations
            #there's an alpha that equals
            alpha[t][i] = emiss[i][seq[t]] * sum(alpha[i-1][:]*trans[:][i]) 
    return alpha

#not working
def backward(seq, trans, emiss):
    beta = np.zeros((seq.size,len(emiss)))
    beta[seq.size-1][:] = 1
    for i in range(len(trans)): #for all state
        for t in range(seq.size-1-1,0,-1): #for all observations
            beta[t-1][i] = sum( trans[i][:] * emiss[:][seq[t]] * beta[t][:] )
            print(beta[t-1][i])
    return beta


from decimal import Decimal, getcontext
getcontext().prec = 30

def max_pointer(v, trans):
    choices = []
    for pji in trans:
        choices.append(v*pji)
    choices = np.array(choices)
    return np.argmax(choices)

# Algorithme de Viterbi
def viterbi(seq, pi, trans, emiss):    
    best = np.zeros(seq.size)
    options = np.zeros(len(emiss))
    vit = np.zeros((seq.size,len(emiss)))
    vit[0][:] = 1
    for t in range(1,seq.size): #for all observations
        for i in range(len(trans)): #for all state
            #there's a value that equals
            
            #best_j = np.max(vit[t-1][i] * trans[:][i])
            #vit[t][i] = emiss[i][seq[t]] * best_j 
            
            best_j = np.max(vit[t-1][i] * trans[:][i])
            vit[t][i] = emiss[i][seq[t]] * best_j 
            
            ##avoid log of negative number
            #replace = False
            #best_j = np.max(vit[t-1][i] * trans[:][i])
            #if(best_j < 0):
            #    best_j = -best_j
            #    replace = True
            #vit[t][i] = np.log( emiss[i][seq[t]] * best_j )
            #if(replace):
            #    vit[t][i] = -vit[t][i]
            
            
            options[i] = np.argmax(vit[t-1][i] * trans[:][i])
            #options[i] = max_pointer(vit, trans[i])
            
            #print(vit[t][i])
            
        best[t] = np.argmax(options)
    #print(best)
    return np.array(best) , vit

'''
def prob_path( path, pi, trans):    #######prob avec viterbi
    state = int(path[0])
    prob = pi[state]
    for i in range(1,path.size):
        next_ = int(path[i])
        prob *= trans[state][next_]
        state = next_
    return prob
'''

def prob_path(l):
    log_odds = np.exp( l )
    log_odds = log_odds / log_odds.sum()
    return np.prod(log_odds)

path = viterbi(sample, pi0, Pij, Eij)
path, l_probas = viterbi(sample, pi0, Pij, Eij)
prob = prob_path( np.array(l_probas) )

print(path)
print(prob) #très peu probable

#on compare et on compte tous les états cachés identiques.
same = np.where(states==path, 1, 0)

print(same.sum()/path.size) # ~60% correct 

[0. 1. 1. ... 0. 0. 0.]
0.0
0.652


<b>Exercice 3</b>: <u>Estimation des paramètres</u>
<br>
3.1) Ecrire une fonction qui utilise tous les résultats de la simulation
(états et symboles) pour compter les nombres d'occurrence $N_{ij}$ est $M_{iO}$ définis
en cours. Estimer $p_{ij}$ est $e_i(O)$, voir slides  37-39 dans la presentation. Attention, pour eviter les probabilites à zero nous alons utiliser les pseudo-count.

In [4]:
template = np.array([[ "00", "01"],[ "10", "11"]])

# Estimation de Parametres par contage
def counting(seq, char):
    #pseudo count
    comb_of_interest = np.ones( template[char].size )
    prev = None
    for c in seq:
        if(prev == char):
            key = str(prev)+str(c)
            ind = np.where(template[char] == key)
            comb_of_interest[ind] += 1
        prev = c
    return comb_of_interest/comb_of_interest.sum()


def estimate(seq):
    #we know there's exactly 2 options
    count1 = counting(seq, 0)
    count2 = counting(seq, 1)
    
    return np.vstack((count1, count2))
    
    
#the sample haven't been transformed to characters.
hid_estimate = estimate(states)
obs_estimate = estimate(sample)

print(hid_estimate)
print(obs_estimate)

[[0.99288061 0.00711939]
 [0.07344633 0.92655367]]
[[0.54553991 0.45446009]
 [0.51599147 0.48400853]]


3.2) <u> Viterbi training </u>: Ecrire une fonction qui utilise 
seulement la séquence $(O_t)_{t=0:T}$ (2emme colone de la simulation) pour estimer les 
paramètres $p_{ij}$ est $e_i(O)$. On s'arretera quand les diferences entre les logVraissamblance est inferieur à 1e-04. Comparer les résultats de 3.1 et de 3.2 (3.2 avec plusieurs restarts,
et avec initialisation des paramètres alèatoire).


In [162]:
import matplotlib.pyplot as plt
delta = 1e-04

# Initialisation aleatoire de Pij, Eij, pi0
def rand_param():
    Pij = []
    one = random.random()
    two = 1 - one
    Pi = [one, two]
    Pij.append(Pi)
    one = random.random()
    two = 1 - one
    Pi = [one, two]
    Pij.append(Pi)
    Pij = np.array(Pij)

    Eij = []
    one = random.random()
    two = 1 - one
    Ei = [one, two]
    Eij.append(Ei)
    one = random.random()
    two = 1 - one
    Ei = [one, two]
    Eij.append(Ei)
    Eij = np.array(Eij)

    one = random.random()
    two = 1 - one
    pi0 = [one, two]
    
    return pi0, Pij, Eij

# Calcule log Vraissamblance
def log_likelihood( sample, path, pi, trans, emiss):
    state = int(path[0])
    obs = sample[0]
    prob = np.log(pi[state] * emiss[state][obs])
    for i in range(1,sample.size):
        obs = sample[i]
        next_ = int(path[i])
        prob += np.log(trans[state][next_] * emiss[state][obs])
        state = next_
    return prob

# Viterbi Training
def viterbi_train(sample, pi, trans, emiss, tresh = delta, max_iter = 50):
    #first iter to get a log likelihood
    path, devnull = viterbi(sample, pi, trans, emiss)
    previous_step = log_likelihood(sample, path, pi, trans, emiss)
    
    path = np.array(list(map(int, path)))
    Pij = estimate(path)
    Eij = estimate(sample) # won't and shouldn't be modified afterward.     
    
    #iteration
    for i in range(max_iter):
        #print(i)
        
        path, devnull = viterbi(sample, pi, trans, emiss)
        like = log_likelihood(sample, path, pi, Pij, Eij)
        
        #if we are close enough we stop
        if(abs(abs(previous_step)-abs(like)) <= tresh):
            return Pij, Eij
        previous_step = like 
        
        path = np.array(list(map(int, path)))
        Pij = estimate(path)
        #Eij = estimate(sample) # The sample won't and shouldn't be modified.     
    return Pij, Eij

pi0, Pij, Eij = rand_param()
Pij, Eij = viterbi_train(sample, pi0, Pij, Eij)
print()
print(Pij)
print(Eij)

#On est vraiment très proche des valeurs obtenues par counting,
#Mais par moment subit de grande variation

#Met toujours 2 itérations ???


[[0.84677419 0.15322581]
 [0.99625468 0.00374532]]
[[0.5682243  0.4317757 ]
 [0.49624866 0.50375134]]


3.3) <u>Viterbi training deuxiemme version </u> Ecrivez une version de 3.3 qui:
- part plusieurs fois (100x) d'une initialisation aléatoire des 
paramètres de l'HMM,
- utilise Viterbi training pour estimer les paramètres,
- calcule la log-vraisemblance pour les paramètres estimés,
- sauvegarde seulement l'estimation avec la valeur maximale de la
log-vraisemblance.

Qu'est-ce que vous observez?



In [163]:
######part plusieurs fois (100x) d'une initialisation aléatoire des paramètres de l'HMM ????
##faire 100 itérations avec viterbi train avec params initiaux différent


# Viterbi Training  deuxiemme version
def viterbi_train2(sample, pi, trans, emiss, max_iter = 100):
    
    #first iter to get a log likelihood
    path, devnull = viterbi(sample, pi, trans, emiss)
    best_log = log_likelihood(sample, path, pi, trans, emiss)
    
    path = np.array(list(map(int, path)))
    Pij = estimate(path)
    Eij = estimate(sample) # how is it useful ? The sample won't and shouldn't be modified.     
    best_param = [Pij, Eij]
    
    #iteration
    for i in range(max_iter):
        
        pi0, Pij, Eij = rand_param()
        Pij, Eij = viterbi_train(sample, pi0, Pij, Eij)
        path, devnull = viterbi(sample, pi, Pij, Eij)
        like = log_likelihood(sample, path, pi, Pij, Eij)
                
        if(like > best_log):
            best_log = like
            best_param = [Pij, Eij]
            
    return best_param[0], best_param[1]

pi0, Pij, Eij = rand_param()
Pij, Eij = viterbi_train2(sample, pi0, Pij, Eij)
print()
print(Pij)
print(Eij)

#on a les mêmes valeurs de façon constante cette fois. 
#mais on s'est un peu éloigné des valeurs de counting. 


[[0.73291139 0.26708861]
 [0.99527187 0.00472813]]
[[0.5682243  0.4317757 ]
 [0.49624866 0.50375134]]
